## 构建本地数据库

In [ ]:
! pip install pymilvus

In [1]:
#  要创建本地 Milvus 矢量数据库，只需MilvusClient通过指定一个文件名来实例化一个来存储所有数据，例如“milvus_demo.db”。

from pymilvus import MilvusClient

client = MilvusClient("milvus_demo.db")

# 生成了.milvus_demo.db.lock 和 milvus_demo.db


## 数据存储器

## 检索方式